In [6]:
!pip install openai
!pip install chromadb
!pip install pypdf
!pip install langchain

In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00


In [14]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from openai import OpenAI
from chromadb.utils import embedding_functions
from pypdf import PdfReader
import os

In [ ]:
from google.colab import userdata
openai_key= userdata.get('API_Key')

In [ ]:
reader =PdfReader()
pdf_texts= [p.extract_text().strip() for o in reader.pages]
pdf_texts=[text for text in pdf_texts if text]

In [7]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
    )

In [ ]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len,
)

character_split_texts = character_splitter.split_text("\n\n".join(pdf_texts))

In [ ]:
token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=0,
    tokens_per_chunk= 256
)

In [ ]:
token_splits_texts = []
for text in character_split_texts:
  token_splits_texts += token_splitter.split_text(text)


In [15]:
embedding_function = SentenceTransformerEmbeddingFunction()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#initialize the Chroma client with persistence

chroma_client = chromadb.PersistentClient(path="db")
collection_name="my_collection"
collection=chroma_client.get_or_create_collection(name=collection_name,embedding_function=embedding_function)

In [16]:
#extracting the embeddings of the token_split_texts

ids= [str(i) for i in range(len(token_splits_texts))]

In [ ]:
collection.add(ids=ids,documents=token_splits_texts)
collection.count()

In [ ]:
client = OpenAI(api_key=openai_key)

In [ ]:
resp = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "What is human life expectancy in the United States?",
        },
    ],
)

In [ ]:
# Function to load docs from a directory


In [ ]:
#Function to generate embeddings using OpenAI API

def get_openai_embeddings(text):
  response = openai.Embedding.create(
    input=text,
    model="text-embedding-ada-002"
  )
  print("+++++Generating embeddings+++++")
  return response['data'][0]['embedding']

In [ ]:
# Generate embeddings for the docs chunks

for doc in chunked_documents:
  print("+++++Generating embeddings+++++")
  embedding = get_openai_embeddings(doc)

In [ ]:
print(doc["embbedding"])

In [ ]:
# Function to query documents
